We use equal weighting on the 3 themes and geometric within the themes to prevent single indicator winner.
Otherwise countries like Israel which has a high percentage of R&D in GDP would have a higher score than they should. We are looking for a technically balanced and advanced target. We also use exclusive percentile to map the values between 0 and 100.

In [1]:
import pandas as pd
import numpy as np

# 1. Load your already-normalized Z-score data
df_z = pd.read_csv("tech_adoption_zscore.csv", index_col=0)

# 2. Define thematic groups and equal weights
groups = {
    'infrastructure_access': [
        'Internet_Users_Pct',
        'Fixed_Broadband_per_100',
        'Mobile_Cellular_per_100'
    ],
    'innovation_capability': [
        'R&D_Expenditure_Pct_GDP',
        'High_Technology_Exports_USD',
        'ICT_Graduates_Pct'
    ],
    'policy_readiness': [
        'Problem_Solving_Tech_Level3_Pct',
        'B2C_Index_2020',
        'DAI_Business',
        'DAI_People',
        'DAI_Gov'
    ]
}
weights = { theme: 1/3 for theme in groups }

# 3. Compute geometric sub-indices directly on the Z-score data
epsilon = 1e-6
sub_df = pd.DataFrame(index=df_z.index)
for theme, cols in groups.items():
    block = df_z[cols]
    # Shift into positive domain
    shift = (-block.min()) + epsilon
    block_pos = block + shift
    # Geometric mean
    sub_df[f'{theme}_sub'] = np.exp(np.log(block_pos).mean(axis=1))

# 4. Build raw composite index via weighted arithmetic mean
raw_index = sum(sub_df[f'{theme}_sub'] * w for theme, w in weights.items())

# 5. Compute exclusive percentile rank (strictly between 0 and 100)
ranks = raw_index.rank(method='average')
N = len(raw_index)
exclusive_pct = (ranks - 0.5) / N * 100

# 6. Compile final results
final = pd.DataFrame({
    **sub_df,
    'raw_index':      raw_index,
    'pct_exclusive':  exclusive_pct
}, index=df_z.index)

# 7. Display or save
print(final.sort_values('raw_index', ascending=False))
# final.to_csv("tech_adoption_composite_indices.csv")


               infrastructure_access_sub  innovation_capability_sub  \
Country Name                                                          
Germany                         2.472855                   2.125601   
Sweden                          2.584540                   1.164556   
Netherlands                     2.325592                   1.075900   
Denmark                         3.027585                   0.807883   
Finland                         2.172224                   0.702888   
United States                   1.825147                   1.936191   
Norway                          2.573868                   0.370539   
Estonia                         2.697526                   0.397279   
Israel                          2.018307                   1.398482   
Austria                         1.737295                   0.942897   
New Zealand                     2.154598                   0.009132   
Australia                       1.939469                   0.411366   
Sloven